<a href="https://colab.research.google.com/github/Abishekdevs/trinity/blob/main/work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps
!pip install -q -U google-generativeai
!pip install --upgrade algoliasearch

In [10]:
#@title 📍 AI Restaurant Explorer (V3 - Stable) { run: "auto" }
LOCATION_INPUT = "Austin, TX, USA" #@param {type:"string"}

import requests, json
from google.colab import userdata

def debug_geocode():
    key = userdata.get('MAPS_API_KEY')
    # Using a slightly different URL format for better compatibility
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": LOCATION_INPUT,
        "key": key
    }

    print(f"--- Debugging Connection ---")
    try:
        response = requests.get(url, params=params)
        data = response.json()

        status = data.get("status")
        error_msg = data.get("error_message", "No error message provided.")

        if status == "OK":
            results = data["results"][0]["geometry"]["location"]
            print(f"✅ Success! Found: {results}")
            return results
        else:
            print(f"❌ Status: {status}")
            print(f"❌ Details: {error_msg}")

            if status == "REQUEST_DENIED":
                print("\n💡 ACTION REQUIRED:")
                print("1. Ensure 'Geocoding API' is ENABLED in Cloud Console.")
                print("2. Ensure 'Billing' is linked to this specific project.")
                print("3. Ensure 'Application Restrictions' is set to 'None' in API Key settings.")
    except Exception as e:
        print(f"❌ Critical Error: {e}")

debug_geocode()

--- Debugging Connection ---
✅ Success! Found: {'lat': 30.267153, 'lng': -97.7430608}


{'lat': 30.267153, 'lng': -97.7430608}

In [ ]:
# 1. INSTALL LIBRARIES
!pip install --upgrade -q algoliasearch gspread google-auth

#@title 📍 AI Advanced Market Intelligence Explorer (v18)
LOCATION_INPUT = "Madurai, tamil Nadu, India" #@param {type:"string"}
RADIUS_METERS = 15500 #@param {type:"slider", min:500, max:50000, step:500}
WORKBOOK_NAME = "Updated Advanced Market Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread

# UPDATED: Import the Synchronous version of the Algolia client
from algoliasearch.search.client import SearchClientSync

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Configure Gemini and Maps
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')

    # UPDATED: Initialize Algolia SearchClientSync
    algolia_client = SearchClientSync(
        userdata.get('ALGOLIA_APP_ID'),
        userdata.get('ALGOLIA_API_KEY')
    )
    ALG_INDEX_NAME = userdata.get('ALGOLIA_INDEX_NAME')

    # Select Gemini Model
    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')

    print("✅ All services (Gemini, Maps, Algolia Sync) Initialized Successfully.")
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 14)
    if not ws_rest.get_all_values():
        ws_rest.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'usp', 'market_gap', 'social_links'])

    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)
    if not ws_menu.get_all_values():
        ws_menu.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Research the restaurant: '{restaurant_name}' at '{location}'.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except:
        return None

def run_app():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)
    existing_rows = ws_rest.get_all_values()
    existing_entries = {(row[2].lower(), row[4].lower()) for row in existing_rows[1:]}

    # 1. Geocoding
    res_geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                           params={"address": LOCATION_INPUT, "key": MAPS_API_KEY}).json()
    if res_geo['status'] != 'OK': return print("❌ Geocoding Error.")
    lat, lng = res_geo['results'][0]['geometry']['location'].values()

    # 2. Nearby Search
    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }
    data = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 20,
        "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
    }
    nearby_data = requests.post(url, headers=headers, json=data).json().get('places', [])

    unique_nearby = [p for p in nearby_data if (p['displayName']['text'].lower(), p['formattedAddress'].lower()) not in existing_entries]
    if not unique_nearby:
        return print(f"✅ No new restaurants found.")

    print(f"🔍 Auditing {len(unique_nearby)} new restaurants...")
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for p in unique_nearby:
        name = p['displayName']['text']
        loc = p['formattedAddress']
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))

        print(f"📊 Analyzing: {name}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        rest_uuid = str(uuid.uuid4())
        d = ai.get('details', {})
        menu_items = ai.get('menu', []) # Extract menu data for dual-saving

        # 3. SAVE TO GOOGLE SHEETS
        ws_rest.append_row([
            curr_time, rest_uuid, name, phone, loc, city, state, country,
            d.get('cuisine'), d.get('website'), d.get('usp'), d.get('market_gap'),
            ", ".join(d.get('social_links', []))
        ])

        # 4. SYNC TO ALGOLIA (Including Menu Data)
        try:
            algolia_record = {
                "objectID": rest_uuid,
                "name": name,
                "location": loc,
                "city": city,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "market_gap": d.get('market_gap'),
                # ADDED: Include the menu as an array within the record
                "menu": menu_items
            }
            # Synchronous call (no await needed)
            algolia_client.save_object(index_name=ALG_INDEX_NAME, body=algolia_record)
            print(f"   ✅ Data + Menu synced to Algolia.")
        except Exception as alg_e:
            print(f"   ⚠️ Algolia Sync Failed: {alg_e}")

        # 5. SAVE MENU ITEMS TO SHEETS (Existing logic)
        menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in menu_items]
        if menu_rows: ws_menu.append_rows(menu_rows)

        time.sleep(2) # Avoid rate limits

    print(f"\n✨ Batch Complete!")

run_app()

In [ ]:
# 1. INSTALL LIBRARIES
!pip install --upgrade -q algoliasearch gspread google-auth

#@title 📍 AI Advanced Market Intelligence Explorer (v20)
LOCATION_INPUT = "Madurai, tamil Nadu, India" #@param {type:"string"}
SEARCH_MODE = "dishes" #@param ["restaurant", "dishes"]
SEARCH_QUERY = "Mutton" #@param {type:"string"}
RADIUS_METERS = 37500 #@param {type:"slider", min:500, max:50000, step:500}
WORKBOOK_NAME = "Updated Advanced Market Intelligence" #@param {type:"string"}

import requests, json, time, uuid
import google.generativeai as genai
from google.colab import userdata, auth
from google.auth import default
import gspread
from algoliasearch.search.client import SearchClientSync

# Initialization
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # FIXED: Using underscores for secret names to prevent whitespace errors
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')

    # Initialize Synchronous Algolia Client
    algolia_client = SearchClientSync(
        userdata.get('ALGOLIA_APP_ID'),
        userdata.get('ALGOLIA_API_KEY')
    )
    ALG_INDEX_NAME = userdata.get('ALGOLIA_INDEX_NAME')

    models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods and 'flash' in m.name]
    model = genai.GenerativeModel(sorted(models)[-1] if models else 'gemini-1.5-flash')

    print(f"✅ Services Initialized. Searching for '{SEARCH_QUERY}' in {LOCATION_INPUT}...")
except Exception as e:
    print(f"❌ Setup Error: {e}")

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    # Ensure Restaurants sheet has all 13 fields [cite: 38-43, 141-147]
    try:
        ws_rest = sh.worksheet("Restaurants")
    except gspread.WorksheetNotFound:
        ws_rest = sh.add_worksheet("Restaurants", 1000, 13)
    if not ws_rest.get_all_values():
        ws_rest.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'usp', 'market_gap', 'social_links'])

    # Ensure Menus sheet has all 7 fields [cite: 49-52, 146-147]
    try:
        ws_menu = sh.worksheet("Menus")
    except gspread.WorksheetNotFound:
        ws_menu = sh.add_worksheet("Menus", 10000, 7)
    if not ws_menu.get_all_values():
        ws_menu.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Research the restaurant: '{restaurant_name}' at '{location}'.\n"
        "Return ONLY JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        clean_text = text[text.find('{'):text.rfind('}') + 1]
        return json.loads(clean_text)
    except: return None

def run_app():
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)
    existing_rows = ws_rest.get_all_values()
    # Deduplication logic using name and location [cite: 119-120]
    existing_entries = {(row[2].lower(), row[4].lower()) for row in existing_rows[1:]}

    # 1. Geocoding
    res_geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                           params={"address": LOCATION_INPUT, "key": MAPS_API_KEY}).json()
    if res_geo['status'] != 'OK': return print("❌ Geocoding Error.")
    lat, lng = res_geo['results'][0]['geometry']['location'].values()

    # 2. Places Search (Targeted Mode)
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }

    if SEARCH_MODE == "dishes":
        url = "https://places.googleapis.com/v1/places:searchText"
        data = {
            "textQuery": f"{SEARCH_QUERY} in {LOCATION_INPUT}",
            "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
        }
    else:
        url = "https://places.googleapis.com/v1/places:searchNearby"
        data = {
            "includedTypes": ["restaurant"],
            "maxResultCount": 20,
            "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
        }

    nearby_data = requests.post(url, headers=headers, json=data).json().get('places', [])
    unique_nearby = [p for p in nearby_data if (p['displayName']['text'].lower(), p['formattedAddress'].lower()) not in existing_entries]

    if not unique_nearby: return print(f"✅ No new entries found for '{SEARCH_QUERY}'. Try a different query or location.")

    print(f"🔍 Found {len(unique_nearby)} new restaurants for '{SEARCH_QUERY}'. Starting audit...")
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for p in unique_nearby:
        name = p['displayName']['text']
        loc = p['formattedAddress']
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))

        print(f"📊 Analyzing: {name}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        rest_uuid = str(uuid.uuid4())
        d = ai.get('details', {})
        menu_items = ai.get('menu', [])

        # 3. SAVE TO GOOGLE SHEETS (13 Fields for Restaurants) [cite: 141-145]
        ws_rest.append_row([
            curr_time, rest_uuid, name, phone, loc, city, state, country,
            d.get('cuisine'), d.get('website'), d.get('usp'), d.get('market_gap'),
            ", ".join(d.get('social_links', []))
        ])

        # SAVE TO GOOGLE SHEETS (7 Fields for Menus) [cite: 146-147]
        menu_rows = [[str(uuid.uuid4()), rest_uuid, m.get('item_name'), m.get('price'), m.get('category'), m.get('is_unique_locally'), m.get('competitive_reasoning')] for m in menu_items]
        if menu_rows: ws_menu.append_rows(menu_rows)

        # 4. SYNC TO ALGOLIA (Bundled Record)
        try:
            algolia_record = {
                "objectID": rest_uuid,
                "name": name,
                "location": loc,
                "city": city,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "menu": menu_items # Unified search by dish name
            }
            algolia_client.save_object(index_name=ALG_INDEX_NAME, body=algolia_record)
            print(f"   ✅ Data + Menu synced to Algolia.")
        except Exception as e: print(f"   ⚠️ Algolia Error: {e}")

        time.sleep(2)

    print(f"\n✨ Batch Complete! Check '{WORKBOOK_NAME}'.")

run_app()

In [ ]:
import gspread
from google.colab import auth, userdata
from google.auth import default
from algoliasearch.search.client import SearchClientSync

# 1. SETUP & AUTHENTICATION
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Initialize Algolia Sync Client
client = SearchClientSync(
    userdata.get('ALGOLIA_APP_ID'),
    userdata.get('ALGOLIA_API_KEY')
)
index_name = userdata.get('ALGOLIA_INDEX_NAME')

def sync_and_verify(workbook_name, sheet_name):
    # 2. FETCH DATA FROM GSHEETS
    sh = gc.open(workbook_name)
    worksheet = sh.worksheet(sheet_name)

    # Get all records
    records = worksheet.get_all_records()
    gsheet_count = len(records)

    if gsheet_count == 0:
        return print("⚠️ No records found in Google Sheets to sync.")

    # 3. SYNC TO ALGOLIA
    print(f"🚀 Syncing {gsheet_count} records to Algolia index '{index_name}'...")
    client.replace_all_objects(index_name=index_name, objects=records)

    # 4. PERFORM COUNT MATCH VERIFICATION (Updated for v4)
    # FIX: Use list_indices() instead of get_indices()
    response = client.list_indices()
    algolia_count = 0

    # Access the 'items' attribute of the response object
    for idx in response.items:
        if idx.name == index_name:
            algolia_count = idx.entries
            break

    # Final Report
    print("-" * 30)
    print(f"📊 SYNC REPORT:")
    print(f"✅ Google Sheets Records: {gsheet_count}")
    print(f"✅ Algolia Index Records: {algolia_count}")

    if gsheet_count == algolia_count:
        print("🟢 VERIFICATION SUCCESS: Record counts match.")
    else:
        print(f"🟡 VERIFICATION PENDING: Found {algolia_count} in Algolia.")
        print("Note: Algolia indexing is asynchronous. If the counts don't match yet, wait 30 seconds and run the count check again.")

# Execute Sync
sync_and_verify("Updated Advanced Market Intelligence", "Restaurants")

In [26]:
# 1. ADDITIONAL IMPORTS
import ipywidgets as widgets
from IPython.display import display, clear_output

# 2. UPDATED RESET LOGIC (Confirmation Button Pattern)
def perform_actual_reset():
    """Logic to actually wipe the data after confirmation."""
    with output_log:
        print("🗑️ Resetting GSheets and Algolia...")
        try:
            # 1. Reset GSheets
            sh = gc.open(WORKBOOK_NAME)
            for sheet_name in ["Restaurants", "Menus"]:
                ws = sh.worksheet(sheet_name)
                ws.clear()
                # Restore headers
                if sheet_name == "Restaurants":
                    ws.append_row(['timestamp', 'id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'usp', 'market_gap', 'social_links'])
                else:
                    ws.append_row(['id', 'rest_id', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

            # 2. Clear Algolia Index
            algolia_client.clear_objects(index_name=ALG_INDEX_NAME)
            print("✨ Everything Reset! Sheets cleared and Algolia index emptied.")
        except Exception as e:
            print(f"❌ Reset Error: {e}")

def on_reset_clicked(b):
    """Triggers the confirmation UI."""
    with output_log:
        clear_output()
        print("⚠️ ARE YOU SURE? This will wipe all data in GSheets and Algolia.")

        # Create confirmation buttons
        confirm_btn = widgets.Button(description="YES, DELETE ALL", button_style='danger')
        cancel_btn = widgets.Button(description="NO, CANCEL", button_style='warning')

        def handle_confirm(b):
            clear_output()
            perform_actual_reset()

        def handle_cancel(b):
            clear_output()
            print("❌ Reset cancelled.")

        confirm_btn.on_click(handle_confirm)
        cancel_btn.on_click(handle_cancel)
        display(widgets.HBox([confirm_btn, cancel_btn]))

# 3. EXISTING SYNC LOGIC
def on_sync_clicked(b):
    with output_log:
        clear_output()
        print("🔄 Starting Full Sync: GSheets -> Algolia...")
        try:
            sh = gc.open(WORKBOOK_NAME)
            records = sh.worksheet("Restaurants").get_all_records()
            if not records:
                print("⚠️ No data in 'Restaurants' sheet to sync.")
                return
            algolia_client.replace_all_objects(index_name=ALG_INDEX_NAME, objects=records)
            res = algolia_client.list_indices()
            count = next((idx.entries for idx in res.items if idx.name == ALG_INDEX_NAME), 0)
            print(f"✅ Sync Complete! Total records in Algolia: {count}")
        except Exception as e:
            print(f"❌ Sync Error: {e}")

# 4. INITIALIZE UI
btn_sync = widgets.Button(description="🔄 Sync Now", button_style='info')
btn_reset = widgets.Button(description="🗑️ Full Reset", button_style='danger')
output_log = widgets.Output()

btn_sync.on_click(on_sync_clicked)
btn_reset.on_click(on_reset_clicked)

print("\n--- 🛠️ CONTROL PANEL ---")
display(widgets.HBox([btn_sync, btn_reset]), output_log)